# Implementing a Route Planner
In this project you will use A\* search to implement a "Google-maps" style route planning algorithm.

## The Map

In [1]:
# Run this cell first!

from helpers import Map, load_map_10, load_map_40, load_map_15, show_map
import math

%load_ext autoreload
%autoreload 

### Map Basics

In [2]:
map_10 = load_map_10()
show_map(map_10)

/home/shreya/.local/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:372: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


/home/shreya/.local/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:426: DeprecationWarning:

plotly.graph_objs.Marker is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Marker
  - plotly.graph_objs.histogram.selected.Marker
  - etc.


/home/shreya/.local/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:39: DeprecationWarning:

plotly.graph_objs.Data is deprecated.
Please replace it with a list or tuple of instances of the following types
  - plotly.graph_objs.Scatter
  - plotly.graph_objs.Bar
  - plotly.graph_objs.Area
  - plotly.graph_objs.Histogram
  - etc.


/home/shreya/.local/lib/python3.6/site-packages

The map above (run the code cell if you don't see it) shows a disconnected network of 10 intersections. The two intersections on the left are connected to each other but they are not connected to the rest of the road network. This map is quite literal in its expression of distance and connectivity. On the graph above, the edge between 2 nodes(intersections) represents a literal straight road not just an abstract connection of 2 cities.

These `Map` objects have two properties you will want to use to implement A\* search: `intersections` and `roads`

**Intersections**

The `intersections` are represented as a dictionary. 

In this example, there are 10 intersections, each identified by an x,y coordinate. The coordinates are listed below. You can hover over each dot in the map above to see the intersection number.

In [3]:
map_10.intersections

{0: (0.7798606835438107, 0.6922727646627362),
 1: (0.7647837074641568, 0.3252670836724646),
 2: (0.7155217893995438, 0.20026498027300055),
 3: (0.7076566826610747, 0.3278339270610988),
 4: (0.8325506249953353, 0.02310946309985762),
 5: (0.49016747075266875, 0.5464878695400415),
 6: (0.8820353070895344, 0.6791919587749445),
 7: (0.46247219371675075, 0.6258061621642713),
 8: (0.11622158839385677, 0.11236327488812581),
 9: (0.1285377678230034, 0.3285840695698353)}

**Roads**

The `roads` property is a list where `roads[i]` contains a list of the intersections that intersection `i` connects to.

In [4]:
# this shows that intersection 0 connects to intersections 7, 6, and 5
map_10.roads[0] 

[7, 6, 5]

In [5]:
# This shows the full connectivity of the map
map_10.roads

[[7, 6, 5],
 [4, 3, 2],
 [1, 4, 3],
 [1, 2, 5, 4],
 [1, 2, 3],
 [0, 3, 7],
 [0],
 [0, 5],
 [9],
 [8]]

In [6]:
# map_40 is a bigger map than map_10
map_40 = load_map_40()
show_map(map_40)

/home/shreya/.local/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:372: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


/home/shreya/.local/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:426: DeprecationWarning:

plotly.graph_objs.Marker is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Marker
  - plotly.graph_objs.histogram.selected.Marker
  - etc.


/home/shreya/.local/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:39: DeprecationWarning:

plotly.graph_objs.Data is deprecated.
Please replace it with a list or tuple of instances of the following types
  - plotly.graph_objs.Scatter
  - plotly.graph_objs.Bar
  - plotly.graph_objs.Area
  - plotly.graph_objs.Histogram
  - etc.


/home/shreya/.local/lib/python3.6/site-packages

In [7]:
map_15 = load_map_15()
show_map(map_15)

/home/shreya/.local/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:372: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


/home/shreya/.local/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:426: DeprecationWarning:

plotly.graph_objs.Marker is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Marker
  - plotly.graph_objs.histogram.selected.Marker
  - etc.


/home/shreya/.local/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:39: DeprecationWarning:

plotly.graph_objs.Data is deprecated.
Please replace it with a list or tuple of instances of the following types
  - plotly.graph_objs.Scatter
  - plotly.graph_objs.Bar
  - plotly.graph_objs.Area
  - plotly.graph_objs.Histogram
  - etc.


/home/shreya/.local/lib/python3.6/site-packages

In [8]:
map_15.intersections

{0: (0.7798606835438107, 0.6922727646627362),
 1: (0.7647837074641568, 0.3252670836724646),
 2: (0.7155217893995438, 0.20026498027300055),
 3: (0.04580558670435442, 0.5886703168399895),
 4: (0.8325506249953353, 0.02310946309985762),
 5: (0.49016747075266875, 0.5464878695400415),
 6: (0.6691919587749445, 0.8820353070895344),
 7: (0.46247219371675075, 0.6258061621642713),
 8: (0.11622158839385677, 0.11236327488812581),
 9: (0.1285377678230034, 0.3285840695698353)}

In [9]:
map_15.roads[0] 

[1, 5, 6, 7]

In [10]:
map_15.roads

[[1, 5, 6, 7],
 [0, 2, 4, 5],
 [1, 3, 4],
 [2, 5, 6, 7, 9],
 [1, 2, 8, 9],
 [0, 1, 3, 7],
 [0, 3, 7],
 [0, 3, 5, 6],
 [4, 9],
 [3, 4, 8]]

### Advanced Visualizations

The map above shows a network of roads which spans 40 different intersections (labeled 0 through 39). 

The `show_map` function which generated this map also takes a few optional parameters which might be useful for visualizaing the output of the search algorithm you will write.

* `start` - The "start" node for the search algorithm.
* `goal`  - The "goal" node.
* `path`  - An array of integers which corresponds to a valid sequence of intersection visits on the map.

In [11]:
show_map(map_40, start=5, goal=34)

/home/shreya/.local/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:372: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


/home/shreya/.local/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:426: DeprecationWarning:

plotly.graph_objs.Marker is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Marker
  - plotly.graph_objs.histogram.selected.Marker
  - etc.


/home/shreya/.local/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:39: DeprecationWarning:

plotly.graph_objs.Data is deprecated.
Please replace it with a list or tuple of instances of the following types
  - plotly.graph_objs.Scatter
  - plotly.graph_objs.Bar
  - plotly.graph_objs.Area
  - plotly.graph_objs.Histogram
  - etc.


/home/shreya/.local/lib/python3.6/site-packages

In [12]:
# run this code, note the effect of including the optional
# parameters in the function call.
show_map(map_40, start=8, goal=24, path=[8, 14, 16, 37, 12, 17, 10, 24])

/home/shreya/.local/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:372: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


/home/shreya/.local/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:426: DeprecationWarning:

plotly.graph_objs.Marker is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Marker
  - plotly.graph_objs.histogram.selected.Marker
  - etc.


/home/shreya/.local/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:39: DeprecationWarning:

plotly.graph_objs.Data is deprecated.
Please replace it with a list or tuple of instances of the following types
  - plotly.graph_objs.Scatter
  - plotly.graph_objs.Bar
  - plotly.graph_objs.Area
  - plotly.graph_objs.Histogram
  - etc.


/home/shreya/.local/lib/python3.6/site-packages

## The Algorithm
### Writing your algorithm
The algorithm written will be responsible for generating a `path` like the one passed into `show_map` above. In fact, when called with the same map, start and goal, as above you algorithm should produce the path `[5, 16, 37, 12, 34]`. However you must complete several methods before it will work.

```bash
> PathPlanner(map_40, 5, 34).path
[5, 16, 37, 12, 34]
```

In [13]:
# Do not change this cell
# When you write your methods correctly this cell will execute
# without problems
class PathPlanner():
    """Construct a PathPlanner Object"""
    def __init__(self, M, start=None, goal=None):
        """ """
        self.map = M
        self.start= start
        self.goal = goal
        self.closedSet = self.create_closedSet() if goal != None and start != None else None
        self.openSet = self.create_openSet() if goal != None and start != None else None
        self.cameFrom = self.create_cameFrom() if goal != None and start != None else None
        self.gScore = self.create_gScore() if goal != None and start != None else None
        self.fScore = self.create_fScore() if goal != None and start != None else None
        self.path = self.run_search() if self.map and self.start != None and self.goal != None else None
        
    def get_path(self):
        """ Reconstructs path after search """
        if self.path:
            return self.path 
        else :
            self.run_search()
            return self.path
    
    def reconstruct_path(self, current):
        """ Reconstructs path after search """
        total_path = [current]
        while current in self.cameFrom.keys():
            current = self.cameFrom[current]
            total_path.append(current)
        return total_path
    
    def _reset(self):
        """Private method used to reset the closedSet, openSet, cameFrom, gScore, fScore, and path attributes"""
        self.closedSet = None
        self.openSet = None
        self.cameFrom = None
        self.gScore = None
        self.fScore = None
        self.path = self.run_search() if self.map and self.start and self.goal else None

    def run_search(self):
        """ """
        if self.map == None:
            raise(ValueError, "Must create map before running search. Try running PathPlanner.set_map(start_node)")
        if self.goal == None:
            raise(ValueError, "Must create goal node before running search. Try running PathPlanner.set_goal(start_node)")
        if self.start == None:
            raise(ValueError, "Must create start node before running search. Try running PathPlanner.set_start(start_node)")

        self.closedSet = self.closedSet if self.closedSet != None else self.create_closedSet()
        self.openSet = self.openSet if self.openSet != None else  self.create_openSet()
        self.cameFrom = self.cameFrom if self.cameFrom != None else  self.create_cameFrom()
        self.gScore = self.gScore if self.gScore != None else  self.create_gScore()
        self.fScore = self.fScore if self.fScore != None else  self.create_fScore()

        while not self.is_open_empty():
            current = self.get_current_node()
            if current == self.goal:
                self.path = [x for x in reversed(self.reconstruct_path(current))]
                return self.path
            else:
                self.openSet.remove(current)
                self.closedSet.add(current)

            for neighbor in self.get_neighbors(current):
                if neighbor in self.closedSet:
                    continue    # Ignore the neighbor which is already evaluated.
                if not neighbor in self.openSet:    # Discover a new node
                    self.openSet.add(neighbor)
                # The distance from start to a neighbor
                #the "dist_between" function may vary as per the solution requirements.
                if self.get_tenative_gScore(current, neighbor) >= self.get_gScore(neighbor):
                    continue        # This is not a better path.
                
                # This path is the best until now. Record it!
                self.record_best_path_to(current, neighbor)
        print("No Path Found")
        self.path = None
        return False

Create the following methods:

In [14]:
def create_closedSet(self):
    """ Creates and returns a data structure suitable to hold the set of nodes already evaluated"""
    # TODO: return a data structure suitable to hold the set of nodes already evaluated
    return set()

In [15]:
def create_openSet(self):
    """ Creates and returns a data structure suitable to hold the set of currently discovered nodes 
    that are not evaluated yet. Initially, only the start node is known."""
    if self.start != None:
        # TODO: return a data structure suitable to hold the set of currently discovered nodes 
        # that are not evaluated yet. Make sure to include the start node.
        
        return {self.start}
    
    raise(ValueError, "Must create start node before creating an open set. Try running PathPlanner.set_start(start_node)")

In [16]:
def create_cameFrom(self):
    """Creates and returns a data structure that shows which node can most efficiently be reached from another,
    for each node."""
    # TODO: return a data structure that shows which node can most efficiently be reached from another,
    # for each node. 
    return dict()

In [17]:
def create_gScore(self):
    """Creates and returns a data structure that holds the cost of getting from the start node to that node, for each node.
    The cost of going from start to start is zero."""
    # TODO:  a data structure that holds the cost of getting from the start node to that node, for each node.
    # for each node. The cost of going from start to start is zero. The rest of the node's values should be set to infinity.
    return {self.start : 0}

In [18]:
def create_fScore(self):
    """Creates and returns a data structure that holds the total cost of getting from the start node to the goal
    by passing by that node, for each node. That value is partly known, partly heuristic.
    For the first node, that value is completely heuristic."""
    # TODO:  a data structure that holds the total cost of getting from the start node to the goal
    # by passing by that node, for each node. That value is partly known, partly heuristic.
    # For the first node, that value is completely heuristic. The rest of the node's value should be 
    # set to infinity.
    return {self.start : 0}


In [19]:
def set_map(self, M):
    """Method used to set map attribute """
    self._reset(self)
    self.start = None
    self.goal = None
    self.map = M
    # TODO: Set map to new value. 


In [20]:
def set_start(self, start):
    """Method used to set start attribute """
    self._reset(self)
    # TODO: Set start value. Remember to remove goal, closedSet, openSet, cameFrom, gScore, fScore, 
    # and path attributes' values.
    self.start = start
    self.goal = None
    self.closedSet = None
    self.openSet = None
    self.cameFrom = None
    self.gScore = None
    self.fScore = None
    self.path = None
    

In [21]:
def set_goal(self, goal):
    """Method used to set goal attribute """
    self._reset(self)
    # TODO: Set goal value. 
    self.goal = goal


In [22]:
def get_current_node(self):
    """ Returns the node in the open set with the lowest value of f(node)."""
    # TODO: Return the node in the open set with the lowest value of f(node).
    openSet = list(self.openSet)
    temp = openSet[0]
    for i in openSet:
        self.calculate_fscore(i)
        if self.fScore[i] < self.fScore[temp]:
            temp = i
    return temp
    

In [23]:
def get_neighbors(self, node):
    """Returns the neighbors of a node"""
    # TODO: Return the neighbors of a node
    neighbors = self.map.roads[node]
    
    for neighbor in neighbors:
        if neighbor in self.closedSet:
            continue
        if neighbor in self.openSet:
            continue
        self.cameFrom[neighbor] = node
        self.gScore[node] = self.get_gScore(node)
        
    return neighbors

In [24]:
def get_gScore(self, node):
    """Returns the g Score of a node"""
    current = node
    previous = self.cameFrom[current] if current in self.cameFrom else None
    g_score = 0
    while previous!= None:
        g_score += self.distance(current, previous)
        current = previous 
        previous = self.cameFrom[current] if current in self.cameFrom else None
    self.gScore[node] = g_score
    return g_score
        

In [25]:
def get_tenative_gScore(self, current, neighbor):
    """Returns the tenative g Score of a node"""
    # TODO: Return the g Score of the current node 
    # plus distance from the current node to it's neighbors
    return self.get_gScore(current) + self.distance(current, neighbor)

In [26]:
def is_open_empty(self):
    """returns True if the open set is empty. False otherwise. """
    # TODO: Return True if the open set is empty. False otherwise.
    if len(self.openSet):
        return False
    else :
        return True


In [27]:
def distance(self, node_1, node_2):
    """ Computes the Euclidean L2 Distance"""
    # TODO: Compute and return the Euclidean L2 Distance
    node_1_x,node_1_y = self.map.intersections[node_1]
    node_2_x,node_2_y = self.map.intersections[node_2]
    temp = math.sqrt((node_1_x - node_2_x)**2 + (node_1_y - node_2_y)**2)
    return temp

In [28]:
def heuristic_cost_estimate(self, node):
    """ Returns the heuristic cost estimate of a node """
    # TODO: Return the heuristic cost estimate of a node
    return self.distance(self.goal, node)

In [29]:
def calculate_fscore(self, node):
    """Calculate the f score of a node. """
    # TODO: Calculate and returns the f score of a node. 
    # REMEMBER F = G + H
    f = self.get_gScore(node) + self.heuristic_cost_estimate(node)
    self.fScore[node] = f
    

In [30]:
def record_best_path_to(self, current, neighbor):
    """Record the best path to a node """
    # TODO: Record the best path to a node, by updating cameFrom, gScore, and fScore
    self.cameFrom[neighbor] = current
    self.gScore[neighbor] = self.get_gScore(neighbor)
    self.fScore[neighbor] = self.calculate_fscore(neighbor)

In [31]:
PathPlanner.create_closedSet = create_closedSet
PathPlanner.create_openSet = create_openSet
PathPlanner.create_cameFrom = create_cameFrom
PathPlanner.create_gScore = create_gScore
PathPlanner.create_fScore = create_fScore
PathPlanner.set_map = set_map
PathPlanner.set_start = set_start
PathPlanner.set_goal = set_goal
PathPlanner.get_current_node = get_current_node
PathPlanner.get_neighbors = get_neighbors
PathPlanner.get_gScore = get_gScore
PathPlanner.get_tenative_gScore = get_tenative_gScore
PathPlanner.is_open_empty = is_open_empty
PathPlanner.distance = distance
PathPlanner.heuristic_cost_estimate = heuristic_cost_estimate
PathPlanner.calculate_fscore = calculate_fscore
PathPlanner.record_best_path_to = record_best_path_to

In [32]:
s = 33
g = 31
planner = PathPlanner(map_40, s, g)
path = planner.path

In [33]:
show_map(map_40, start=s, goal=g, path=planner.path)

/home/shreya/.local/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:372: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


/home/shreya/.local/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:426: DeprecationWarning:

plotly.graph_objs.Marker is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Marker
  - plotly.graph_objs.histogram.selected.Marker
  - etc.


/home/shreya/.local/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:39: DeprecationWarning:

plotly.graph_objs.Data is deprecated.
Please replace it with a list or tuple of instances of the following types
  - plotly.graph_objs.Scatter
  - plotly.graph_objs.Bar
  - plotly.graph_objs.Area
  - plotly.graph_objs.Histogram
  - etc.


/home/shreya/.local/lib/python3.6/site-packages

In [34]:
s = 8
g = 0
planner = PathPlanner(map_15, s, g)
path = planner.path

In [35]:
show_map(map_15, start=s, goal=g, path=planner.path)

/home/shreya/.local/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:372: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


/home/shreya/.local/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:426: DeprecationWarning:

plotly.graph_objs.Marker is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Marker
  - plotly.graph_objs.histogram.selected.Marker
  - etc.


/home/shreya/.local/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:39: DeprecationWarning:

plotly.graph_objs.Data is deprecated.
Please replace it with a list or tuple of instances of the following types
  - plotly.graph_objs.Scatter
  - plotly.graph_objs.Bar
  - plotly.graph_objs.Area
  - plotly.graph_objs.Histogram
  - etc.


/home/shreya/.local/lib/python3.6/site-packages

### Testing your Code
If the code below produces no errors, your algorithm is behaving correctly. You are almost ready to submit! Before you submit, go through the following submission checklist:

**Submission Checklist**

1. Does my code pass all tests?
2. Does my code implement `A*` search and not some other search algorithm?
3. Do I use an **admissible heuristic** to direct search efforts towards the goal?
4. Do I use data structures which avoid unnecessarily slow lookups?

When you can answer "yes" to all of these questions, submit by pressing the Submit button in the lower right!

In [36]:
from test import test

test(PathPlanner)

SyntaxError: invalid syntax (test.py, line 1)

## Questions

**Instructions**  Answer the following questions in your own words. We do not you expect you to know all of this knowledge on the top of your head. We expect you to do research and ask question. However do not merely copy and paste the answer from a google or stackoverflow. Read the information and understand it first. Then use your own words to explain the answer.

- How would you explain A-Star to a family member(layman)?

** ANSWER **:

let us suppose you need to reach from one city to another city via different cities and you have many friends, and you know where your destination is (you have the approx distances from your location to the destination). Now one by one you and your friends started your journey , first of all you and your friends have to cover all neighbouring cities then from there you need to calculate the distance travelled + approx distance remaining and all your friends does the same, now among you and your friends the total calculated distance (i.e travelled + approx) is minimun will proceed towards it's neighbors (consider you have unlimited number of friends) and this process continous , now let us suppose some of your friends have reached the destination city, then the final path will be given by the trajactory of your friend who has travelled the least distance to reach the destination, in real life we do not have this much amout of time to travel and find the shortest path so we just compute them using same algorithm virtaully.

- How does A-Star search algorithm differ from Uniform cost search? What about Best First search?

** ANSWER **:
A star algorithm is advancement to uniform cost search as we apply huristics in it , generally uniform cost search is direction less but in a star we generally approach towards our goal , so it it more optimised solution than uniform cost search . In the case of best first search algorithm we use a priority queue and pure huristics, A star is a type of best first search algorithm which does not uses priority queue.

- What is a heuristic?

** ANSWER **:heuristic is an intelligent way to solve problem by using educated guesses.

- What is a consistent heuristic?

** ANSWER **: consistent heuristic is the heuristic which results are always less than the estimated results for the neighbouring nodes.

- What is a admissible heuristic? 

** ANSWER **:admissible heuristic is the heuristic in which the estimated results are not overestimated or the cost it estimates to reach the goal is lower than the lowest possible cost.

- ___ admissible heuristic are consistent.
*CHOOSE ONE*
    - All
    - Some
    - None
    
** ANSWER **:Some


- ___ Consistent heuristic are admissible.
*CHOOSE ONE*
    - All
    - Some
    - None
    
** ANSWER **:All